In [2]:
import warnings
import pandas as pd
import sys, os


from causalnex.structure import StructureModel


In [3]:
df = pd.read_csv("../data/df_all_encoded.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   game_id             605 non-null    object 
 1   cta_start_x         605 non-null    float64
 2   cta_start_y         605 non-null    float64
 3   cta_end_x           605 non-null    float64
 4   cta_end_y           605 non-null    float64
 5   cta_height          605 non-null    float64
 6   cta_width           605 non-null    float64
 7   color_1_occurance   605 non-null    float64
 8   color_2_occurance   605 non-null    float64
 9   color_3_occurance   605 non-null    float64
 10  color_4_occurance   605 non-null    float64
 11  color_5_occurance   605 non-null    float64
 12  audio_duration(ms)  605 non-null    float64
 13  audio_intensity     605 non-null    float64
 14  audio_frame_count   605 non-null    float64
 15  beg_frame_angry     605 non-null    float64
 16  beg_fram

In [4]:
df.drop(['color_1_occurance',
 "color_2_occurance",
 "color_3_occurance",
 "color_4_occurance",
 'color_5_occurance'],inplace=True, axis=1)

#### Create Structure Model

In [7]:
df.drop(['game_id'],inplace=True,axis=1)

In [12]:
df = df.reset_index()

In [18]:
import numpy as np

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [19]:
df = clean_dataset(df)

/tmp/ipykernel_417206/1366205758.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)


In [30]:
sm = StructureModel()

#### Create Edges
* Transform and encode

In [21]:
import numpy as np

struct_data = df.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

[]


In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

#### Scale and normalize numerical data

In [23]:
from sklearn import preprocessing

x = struct_data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

#### Apply the NOTEARS algorithm to learn the structure.

In [31]:
from causalnex.structure.notears import from_pandas

sm = from_pandas(df)

KeyboardInterrupt: 

#### Visualize

In [ ]:
from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE


viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
    prog='dot',
)
Image(viz.draw(format='png'))